In [1]:
# imports 
import pandas as pd # to read the csv files 
import os.path
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# need to allow for the memory limit to be able to grow (?) https://www.tensorflow.org/guide/gpu
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
# maybe we should use IMG instead...
# thanks so much to Alex Kyllo for pointing this out to us!
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
rescale_datagen = ImageDataGenerator(rescale=1./255)
train_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/train',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)
validation_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/valid',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)

Found 99999 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [ ]:
# this time around, we'll play around with the BN layer order, as well as using 'selu' as our activation function, which requires 
# LeCun initialization 
vgg_v2 = models.Sequential()
vgg_v2.add(layers.InputLayer(input_shape=(256, 256, 3)))
for i in range(2):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(64, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
for i in range(2):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(128, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
for i in range(3):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(256, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
for i in range(3):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Conv2D(512, (3, 3), kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.MaxPooling2D((2, 2)))
vgg_v2.add(layers.Flatten())
for i in range(2):
    vgg_v2.add(layers.BatchNormalization())
    vgg_v2.add(layers.Activation('selu'))
    vgg_v2.add(layers.Dense(4086, kernel_initializer='lecun_normal', use_bias=False))
vgg_v2.add(layers.BatchNormalization())
vgg_v2.add(layers.Dense(2, activation='softmax'))

In [ ]:
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [ ]:
# DO NOT RUN THIS CELL !
vgg_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])
vgg_v2_history = vgg_v2.fit(train_generator, 
                                  epochs=50, 
                                  validation_data=validation_generator, 
                                  callbacks=[
                                      tf.keras.callbacks.ModelCheckpoint('models/vgg_v2', save_best_only=True),
                                      tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True),
                                      tf.keras.callbacks.TensorBoard(get_run_logdir())
                                  ])

In [4]:
test_generator = rescale_datagen.flow_from_directory('data/archive/real_vs_fake/test',
                                                      class_mode='binary',
                                                     batch_size=32, 
                                                     shuffle=True,
                                                     seed=42)

Found 20000 images belonging to 2 classes.


In [5]:
vgg_v2 = tf.keras.models.load_model('models/vgg_v2')

In [28]:
vgg_v2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 256, 256, 3)       12        
_________________________________________________________________
activation (Activation)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 64)      1728      
_________________________________________________________________
batch_normalization_1 (Batch (None, 254, 254, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 64)      36864     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 64)      0

In [47]:
class PrintSomeExamplePredictionsCallback(tf.keras.callbacks.Callback):
    def on_test_batch_end(self, batch, logs):
        print('\n')
        print(self.model.get_layer(name='dense_2').output)
        print('\n')

In [48]:
vgg_v2.evaluate(test_generator, batch_size=32, callbacks=[PrintSomeExamplePredictionsCallback()])



Tensor("strided_slice_87:0", shape=(), dtype=float32)


  1/625 [..............................] - ETA: 0s - loss: 0.1503 - accuracy: 0.9688

Tensor("strided_slice_89:0", shape=(), dtype=float32)


  2/625 [..............................] - ETA: 32s - loss: 0.0751 - accuracy: 0.9844

Tensor("strided_slice_91:0", shape=(), dtype=float32)


  3/625 [..............................] - ETA: 43s - loss: 0.0501 - accuracy: 0.9896

Tensor("strided_slice_93:0", shape=(), dtype=float32)


  4/625 [..............................] - ETA: 48s - loss: 0.0400 - accuracy: 0.9922

Tensor("strided_slice_95:0", shape=(), dtype=float32)


  5/625 [..............................] - ETA: 51s - loss: 0.0324 - accuracy: 0.9937

Tensor("strided_slice_97:0", shape=(), dtype=float32)


  6/625 [..............................] - ETA: 52s - loss: 0.0270 - accuracy: 0.9948

Tensor("strided_slice_99:0", shape=(), dtype=float32)


  7/625 [..............................] - ETA: 54s - loss: 0.0231 - accuracy: 0.9955

 57/625 [=>............................] - ETA: 59s - loss: 0.0121 - accuracy: 0.9967

Tensor("strided_slice_201:0", shape=(), dtype=float32)


 58/625 [=>............................] - ETA: 59s - loss: 0.0119 - accuracy: 0.9968

Tensor("strided_slice_203:0", shape=(), dtype=float32)


 59/625 [=>............................] - ETA: 59s - loss: 0.0119 - accuracy: 0.9968

Tensor("strided_slice_205:0", shape=(), dtype=float32)


 60/625 [=>............................] - ETA: 58s - loss: 0.0117 - accuracy: 0.9969

Tensor("strided_slice_207:0", shape=(), dtype=float32)


 61/625 [=>............................] - ETA: 58s - loss: 0.0117 - accuracy: 0.9969

Tensor("strided_slice_209:0", shape=(), dtype=float32)


 62/625 [=>............................] - ETA: 58s - loss: 0.0116 - accuracy: 0.9970

Tensor("strided_slice_211:0", shape=(), dtype=float32)


 63/625 [==>...........................] - ETA: 58s - loss: 0.0114 - accuracy: 0.9970

Tensor("strided_slice_213:0", shape=(), dtype=flo

KeyboardInterrupt: 